In [ ]:
# import python built-in libraries
import os.path

# import external libraries
import netCDF4 
import pandas as pd
import numpy as np
import yaml
from datetime import datetime, timedelta
from cftime import num2date, date2num

# import user libraries
from utils import description_module
from utils import conversion_module

# load configuration file
with open('config.yml') as f:
    cfg = yaml.load(f, Loader = yaml.FullLoader)

In [ ]:

dataPath = "./data/"
# dataPath = "./raw_data/Maebong_and_Moonam/"
 
files = os.listdir(dataPath)

datasetDict = {}
for idx,nc_type_file in enumerate(files):
    if nc_type_file.split(".")[-1] == 'nc':
        datasetDict[nc_type_file] = netCDF4.Dataset(dataPath+nc_type_file,mode="r")

In [ ]:
# ncfile_name = '20210731_test_example_2.nc'
ncfile_name = '2023_former.nc'
output_dataset = datasetDict[ncfile_name]

# 하단 데이터를 갖고 있습니다.
output_dataset.variables.keys()

#mdww : mean direction of wind waves
#mpww : mean period of wind waves
#mwd : mean wave direction
#mwp : mean wave period
#wmb : Model bathymetry
#dwi : Ocean surface stress equivalent 10m neutral wind direction
#swh : Significant height of combined wind waves and swell
#shww : significant height of wind waves

In [ ]:
for i in output_dataset.variables.keys():
    print("")
    print(output_dataset[i])
    print("")

In [ ]:
output_dataset['latitude']

In [ ]:
output_dataset['p140121'][:].data.shape

In [ ]:
len(output_dataset['time'].get_dims())

In [ ]:
output_dataset['p140121'].long_name

In [ ]:

array_format = np.array(output_dataset['v100']).T.reshape([4344,2,1])

In [ ]:
array_format

In [ ]:
info_dict, meta_dict = conversion_module.netCDF4_to_CSV(output_dataset)
data_df = conversion_module.dict_to_dataframe(info_dict, meta_dict)
data_df = conversion_module.measure_wind_info(data_df)

fname = str(meta_dict['latitude'][0])+"(lat) "+str(meta_dict['longitude'][0])+"(lon) located data"+'.csv'
conversion_module.save_df(data_df, cfg['project_title'], fname)

del info_dict
del meta_dict

In [ ]:
data_df